In [ ]:
!pip install gradio
!pip install datetime
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 10.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import datetime
import matplotlib.pyplot as plt
import os

# --- Define Your Custom Theme ---
custom_theme = gr.themes.Default(
    primary_hue="fuchsia",  # Changes the primary accent color for buttons to fuchsia
    secondary_hue="blue",
    neutral_hue="yellow",
    font=["Georgia", "Times New Roman"],
    font_mono=["Fira Code", "monospace"]
)

# --- Define the food_database ---
food_database = {
    "Apple": 52, "Banana": 89, "Egg": 78,
    "Chicken Breast (100g)": 165, "Rice (100g)": 130, "Steak (100g)": 250,
    "Salmon (100g)": 206, "Oatmeal (100g)": 15, "Avocado": 273,
    "Spinach (100g)": 23, "Custom": 0
}

# --- BMI Calculator ---
def calculate_bmi(height, weight, unit):
    if unit == "Metric (cm, kg)":
        height_m = height / 100
        weight_kg = weight
    else:
        height_m = height * 0.0254
        weight_kg = weight * 0.453592
    bmi = weight_kg / (height_m ** 2)
    if bmi < 18.5:
        category = "Underweight"
    elif bmi < 24.9:
        category = "Normal"
    elif bmi < 29.9:
        category = "Overweight"
    else:
        category = "Obese"
    return round(bmi, 2), category

# --- Daily Metabolic Rate Calculator ---
def calculate_metabolic_rate(age, gender, height, weight, activity):
    if gender == "Male":
        bmr = 88.362 + 13.397 * weight + 4.799 * height - 5.677 * age
    else:
        bmr = 447.593 + 9.247 * weight + 3.098 * height - 4.330 * age
    activity_dict = {
        "Sedentary": 1.2, "Lightly active": 1.375, "Moderately active": 1.55,
        "Very active": 1.725, "Super active": 1.9
    }
    tdee = bmr * activity_dict[activity]
    return round(bmr, 1), round(tdee, 1), f"Calories for maintenance: {int(tdee)} kcal", tdee

# --- Food Tracker ---
def log_food(day_str, food, manual_cal, daily_logs, tdee):
    try:
        day = datetime.datetime.strptime(day_str, "%Y-%m-%d").date()
    except ValueError:
        return 0, [], daily_logs, "Invalid date format.", None
    date_str = day.strftime("%Y-%m-%d")
    if date_str not in daily_logs:
        daily_logs[date_str] = {"total_calories": 0}
    cal = manual_cal if food == "Custom" and manual_cal is not None else food_database.get(food, 0)
    daily_logs[date_str]["total_calories"] += cal
    last_7_days_data = []
    dates = []
    calories = []
    for i in range(7):
        current_date = datetime.date.today() - datetime.timedelta(days=i)
        current_date_str = current_date.strftime("%Y-%m-%d")
        daily_cal = daily_logs.get(current_date_str, {}).get("total_calories", 0)
        last_7_days_data.append([current_date_str, daily_cal])
        dates.append(current_date)
        calories.append(daily_cal)
    sorted_data = sorted(zip(dates, calories))
    sorted_dates = [date.strftime("%Y-%m-%d") for date, _ in sorted_data]
    sorted_calories = [cal for _, cal in sorted_data]
    info_message = ""
    if tdee is not None and isinstance(tdee, (int, float)):
        calorie_difference = daily_logs[date_str]["total_calories"] - tdee
        if calorie_difference > 0:
            info_message = f"Today's intake is {abs(calorie_difference):.1f} calories over your TDEE."
        elif calorie_difference < 0:
            info_message = f"Today's intake is {abs(calorie_difference):.1f} calories under your TDEE."
        else:
            info_message = "Today's intake meets your TDEE."
    else:
        info_message = "TDEE not available for comparison."
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(sorted_dates, sorted_calories, color='skyblue')
    ax.set_ylabel("Calories")
    ax.set_title("Daily Calorie Intake (Last 7 Days)")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.close(fig)  # Close the figure to prevent memory leak
    return daily_logs[date_str]["total_calories"], last_7_days_data, daily_logs, info_message, fig

# --- Gradio UI with gr.Blocks() ---
custom_css = """
body, .gradio-container {
    background-color: #F8C8DC; /* Pastel pink */
}
.gradio-container .gr-box {
    background-color: #FFFACD; /* Pastel yellow for all component boxes */
}
.gr-button {
    border-radius:16px;
    font-weight:bold;
}
"""

with gr.Blocks(theme=custom_theme, css=custom_css) as app:
    gr.Markdown("## 🩺 **BME Health Calculator Suite**")
    daily_logs_state = gr.State({})
    metabolism_tdee_hidden_output = gr.Number(visible=False)

    with gr.Tab("💖 BMI Calculator"):
        gr.Markdown("### BMI Calculator")
        with gr.Row():
            height_input_bmi = gr.Slider(80, 220, value=170, label="Height")
            weight_input_bmi = gr.Slider(30, 150, value=70, label="Weight")
            unit_input_bmi = gr.Radio(["Metric (cm, kg)", "Imperial (inches, lbs)"], value="Metric (cm, kg)", label="Unit System")
        with gr.Row():
            bmi_value_output = gr.Number(label="BMI Value")
            bmi_category_output = gr.Textbox(label="Category")
        calculate_btn_bmi = gr.Button("Calculate BMI")
        calculate_btn_bmi.click(
            fn=calculate_bmi,
            inputs=[height_input_bmi, weight_input_bmi, unit_input_bmi],
            outputs=[bmi_value_output, bmi_category_output]
        )

    with gr.Tab("💖 Metabolic Rate"):
        gr.Markdown("### Metabolic Rate Calculator")
        with gr.Row():
            age_slider = gr.Slider(10, 90, value=25, label="Age")
            gender_radio = gr.Radio(["Male", "Female"], label="Gender")
        with gr.Row():
            height_slider = gr.Slider(100, 220, value=170, label="Height (cm)")
            weight_slider = gr.Slider(30, 150, value=70, label="Weight (kg)")
        activity_dropdown = gr.Dropdown(["Sedentary", "Lightly active", "Moderately active", "Very active", "Super active"], value="Sedentary", label="Activity Level")
        calculate_btn_metabolism = gr.Button("Calculate TDEE")
        with gr.Row():
            bmr_output = gr.Number(label="BMR")
            tdee_display_output = gr.Number(label="TDEE")
        info_output = gr.Textbox(label="Info")
        calculate_btn_metabolism.click(
            fn=calculate_metabolic_rate,
            inputs=[age_slider, gender_radio, height_slider, weight_slider, activity_dropdown],
            outputs=[bmr_output, tdee_display_output, info_output, metabolism_tdee_hidden_output]
        )

    with gr.Tab("💖 Food Tracker"):
        gr.Markdown("### Food Tracker")
        tdee_input = gr.Number(label="Your TDEE (from Metabolic Rate tab)")
        with gr.Row():
            food_date_input = gr.Textbox(value=str(datetime.date.today()), label="Date (YYYY-MM-DD)")
            food_food_input = gr.Dropdown(list(food_database.keys()), label="Food")
            food_manual_cal_input = gr.Number(label="Manual Calories Entry (if Custom)")
        add_food_btn = gr.Button("Add Food")
        with gr.Row():
            today_calories_output = gr.Number(label="Today's Total Calories")
            info_message_output = gr.Textbox(label="Comparison Info")
        last_7_days_df = gr.Dataframe(headers=["Date", "Calories"], label="Last 7 Days")
        plot_output = gr.Plot(label="Daily Calorie Intake Plot")
        add_food_btn.click(
            fn=log_food,
            inputs=[food_date_input, food_food_input, food_manual_cal_input, daily_logs_state, tdee_input],
            outputs=[today_calories_output, last_7_days_df, daily_logs_state, info_message_output, plot_output]
        )
    metabolism_tdee_hidden_output.change(
        fn=lambda tdee: gr.update(value=tdee),
        inputs=metabolism_tdee_hidden_output,
        outputs=tdee_input
    )

app.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>